In [1]:
import pandas as pd
import sasoptpy as so
import random
import string
import os
from subprocess import Popen

In [2]:
data = pd.read_csv("data/fplreview.csv", index_col=0)
data['id'] = data.index + 1
data['price'] = data['Value'] / 10
player_dict = data.set_index('id', drop=True).to_dict(orient='index')

In [3]:
data.head()

,Value,9_Pts,9_xMins,10_Pts,10_xMins,11_Pts,11_xMins,12_Pts,12_xMins,13_Pts,...,16_xMins,17_Pts,17_xMins,18_Pts,18_xMins,Pos,Name,Team,id,price
0,47.0,0.173,4,0.197,5,0.326,7,0.250,7,0.444,...,12,0.586,14,0.706,18,G,Leno,Arsenal,1,4.7
1,40.0,0.000,0,0.000,0,0.000,0,0.000,0,0.000,...,0,0.000,0,0.000,0,G,Rúnarsson,Arsenal,2,4.0
2,63.0,0.000,0,0.000,0,0.000,0,0.000,0,0.000,...,0,0.000,0,0.000,0,M,Willian,Arsenal,3,6.3
3,99.0,5.218,82,4.238,75,5.591,74,3.224,71,5.499,...,70,4.441,68,4.536,72,F,Aubameyang,Arsenal,4,9.9
4,43.0,0.245,6,0.213,6,0.531,11,0.313,12,0.560,...,16,0.717,17,0.636,17,D,Cédric,Arsenal,5,4.3


In [4]:
# Find ID of White
data[data['Name'] == 'White'] # 67

,Value,9_Pts,9_xMins,10_Pts,10_xMins,11_Pts,11_xMins,12_Pts,12_xMins,13_Pts,...,16_xMins,17_Pts,17_xMins,18_Pts,18_xMins,Pos,Name,Team,id,price
66,44.0,3.33,88,2.476,85,3.807,84,1.397,85,3.34,...,78,2.576,72,2.195,73,D,White,Arsenal,67,4.4
598,45.0,0.00,0,0.000,0,0.000,0,0.000,0,0.00,...,0,0.000,0,0.000,0,M,White,Newcastle,599,4.5


In [5]:
# Find ID of Livramento
data[data['Name'] == 'Livramento'] # 491

,Value,9_Pts,9_xMins,10_Pts,10_xMins,11_Pts,11_xMins,12_Pts,12_xMins,13_Pts,...,16_xMins,17_Pts,17_xMins,18_Pts,18_xMins,Pos,Name,Team,id,price
490,42.0,3.593,87,3.069,83,2.967,82,3.251,81,1.117,...,76,2.578,77,3.073,75,D,Livramento,Southampton,491,4.2


In [6]:
def get_random_id(n):
    return ''.join(random.choice(string.ascii_letters + string.digits) for _ in range(n))

def read_solution(m, sol_file):
    with open(sol_file, 'r') as f:
        for v in m.get_variables():
            v.set_value(0)
        for line in f:
            if 'objective value' in line:
                continue
            words = line.split()
            v = m.get_variable(words[1])
            v.set_value(float(words[2]))


def get_solution_info(player_ids, m, gws, xp):
    squad = m.get_variable('squad')
    lineup = m.get_variable('lineup')
    squad_players = []
    lineup_selection = []
    for p in player_ids:
        if squad[p].get_value() > 0.5:
            squad_players.append(p)
    for g in gws:
        for p in squad_players:
            if lineup[p,g].get_value() > 0.5:
                lineup_selection.append({'player': p, 'gw': g, 'xp': round(xp[g][p], 3) })
    return {'squad': squad_players, 'lineup_selection': lineup_selection}


def find_best_rotation(players, N=2, K=1, max_iter=1, first_gw=1, last_gw=38, exclude=[], rules=[]):
    problem_name = get_random_id(10)
    m = so.Model(name=f'pick_{K}_choose_{N}_{problem_name}')
    gameweeks = list(range(first_gw, last_gw+1))
    player_ids = players['id'].tolist()
    squad = m.add_variables(player_ids, vartype=so.binary, name='squad')
    lineup = m.add_variables(player_ids, gameweeks, vartype=so.binary, name='lineup')

    xp = {}
    indexed_players = players.set_index('id')
    for g in gameweeks:
        xp[g] = indexed_players["{}_Pts".format(g)]

    if len(exclude) > 0:
        m.add_constraints((squad[p] == 0 for p in exclude), name='ban_player')

    m.add_constraint(so.expr_sum(squad[p] for p in player_ids) == N, name='select_N')
    m.add_constraints((so.expr_sum(lineup[p, g] for p in player_ids) == K for g in gameweeks), name='pick_K')
    m.add_constraints((lineup[p,g] <= squad[p] for p in player_ids for g in gameweeks), name='valid_picks_only')
    for rule_no, rule in enumerate(rules):
        if rule['bound'] == 'lower':
            m.add_constraint(so.expr_sum(squad[p] for p in rule['ids']) >= rule['value'], name=f'rule_{rule_no}')
        elif rule['bound'] == 'upper':
            m.add_constraint(so.expr_sum(squad[p] for p in rule['ids']) <= rule['value'], name=f'rule_{rule_no}')

    m.set_objective(-so.expr_sum(xp[g][p] * lineup[p,g] for p in player_ids for g in gameweeks), sense='N', name='total_xp')

    solutions = []

    m.export_mps(f"tmp/{problem_name}.mps")
    command = f"cbc tmp/{problem_name}.mps solve solu tmp/{problem_name}.sol"
    Popen(command).wait()
    read_solution(m, f'tmp/{problem_name}.sol')
    sol = get_solution_info(player_ids, m, gameweeks, xp)

    solutions.append(sol)

    os.unlink(f'tmp/{problem_name}.mps')
    os.unlink(f'tmp/{problem_name}.sol')

    for it in range(1, max_iter):
        c = m.add_constraint(so.expr_sum(squad[p] for p in sol['squad']) <= N-1, name=f'cutoff_{it}')
        m.export_mps(f"tmp/{problem_name}.mps")
        Popen(command).wait()
        read_solution(m, f'tmp/{problem_name}.sol')
        sol = get_solution_info(player_ids, m, gameweeks, xp)
        solutions.append(sol)

        os.unlink(f'tmp/{problem_name}.mps')
        os.unlink(f'tmp/{problem_name}.sol')

    return solutions

In [7]:
# Rule: Have at least 1 of Livramento or White
rules = [
    {'bound': 'lower', 'value': 1, 'ids': [67, 491]}
    ]
# Get only defenders under 4.5 M
budget_defenders = data[(data['Pos'] == 'D') & (data['price'] <= 4.5)]
solutions = find_best_rotation(players=budget_defenders, N=2, K=1, max_iter=30, first_gw=9, last_gw=16, exclude=[], rules=rules)

NOTE: Initialized model pick_1_choose_2_JYvQUZQ0ch.


In [8]:
summary = []
for sol_no, sol in enumerate(solutions):
    print(f"Solution {sol_no+1}")
    selected_names = []
    details = []
    cost = 0
    for p in sol['squad']:
        name = player_dict[p]['Name']
        games = [str(sel['gw']) for sel in sol['lineup_selection'] if sel['player'] == p]
        selected_names.append(f"{name} ({len(games)})")
        details.append(f"{name}: [{','.join(games)}]")
        cost += player_dict[p]['price']
    selected_names = ", ".join(selected_names)
    details = ", ".join(details)
    print(f"Players: {selected_names}")
    total = 0
    for sel in sol['lineup_selection']:
        print(f"GW{sel['gw']:<2d} {player_dict[sel['player']]['Name']:12s} xP: {sel['xp']}")
        total += sel['xp']
    print(f"Total xP: {total:.2f}")
    summary.append({'name': selected_names, 'cost': cost, 'total': total, 'xp_per_cost': total/cost, 'details': details})

Solution 1
Players: Henry (3), Livramento (5)
GW9  Livramento   xP: 3.593
GW10 Livramento   xP: 3.069
GW11 Henry        xP: 4.108
GW12 Livramento   xP: 3.251
GW13 Henry        xP: 3.114
GW14 Livramento   xP: 2.705
GW15 Livramento   xP: 2.956
GW16 Henry        xP: 3.495
Total xP: 26.29
Solution 2
Players: White (3), Livramento (5)
GW9  Livramento   xP: 3.593
GW10 Livramento   xP: 3.069
GW11 White        xP: 3.807
GW12 Livramento   xP: 3.251
GW13 White        xP: 3.34
GW14 Livramento   xP: 2.705
GW15 Livramento   xP: 2.956
GW16 White        xP: 3.083
Total xP: 25.80
Solution 3
Players: White (5), Coady (3)
GW9  White        xP: 3.33
GW10 Coady        xP: 3.255
GW11 White        xP: 3.807
GW12 Coady        xP: 2.767
GW13 White        xP: 3.34
GW14 Coady        xP: 3.271
GW15 White        xP: 2.484
GW16 White        xP: 3.083
Total xP: 25.34
Solution 4
Players: White (3), Salisu (5)
GW9  Salisu       xP: 3.532
GW10 Salisu       xP: 3.054
GW11 White        xP: 3.807
GW12 Salisu       xP: 3.

In [9]:
pd.DataFrame(summary)

,name,cost,total,xp_per_cost,details
0,"Henry (3), Livramento (5)",8.7,26.291,3.021954,"Henry: [11,13,16], Livramento: [9,10,12,14,15]"
1,"White (3), Livramento (5)",8.6,25.804,3.000465,"White: [11,13,16], Livramento: [9,10,12,14,15]"
2,"White (5), Coady (3)",8.9,25.337,2.846854,"White: [9,11,13,15,16], Coady: [10,12,14]"
3,"White (3), Salisu (5)",8.9,25.314,2.844270,"White: [11,13,16], Salisu: [9,10,12,14,15]"
4,"Ajer (3), Livramento (5)",8.7,25.281,2.905862,"Ajer: [11,13,16], Livramento: [9,10,12,14,15]"
5,"White (5), Lowton (3)",8.8,25.174,2.860682,"White: [9,11,13,15,16], Lowton: [10,12,14]"
6,"White (3), Henry (5)",8.9,24.649,2.769551,"White: [9,13,15], Henry: [10,11,12,14,16]"
7,"White (5), Taylor (3)",8.8,24.561,2.791023,"White: [9,11,13,15,16], Taylor: [10,12,14]"
8,"Coady (3), Livramento (5)",8.7,24.408,2.805517,"Coady: [10,13,14], Livramento: [9,11,12,15,16]"
9,"Duffy (3), Livramento (5)",8.6,24.375,2.834302,"Duffy: [11,13,16], Livramento: [9,10,12,14,15]"
